# LIF comparison

## setup

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn
import matplotlib

filenames = [
    ["Exact", "lif_exact.csv"],
    ['Lava (CPU)', 'lif_lava_cpu_float.csv'],
    ['Nengo', 'lif_nengo.csv'],
    ['Norse', 'lif_norse.csv'],
    ['Rockpool', 'lif_rockpool.csv'],
    ['Sinabs', 'lif_sinabs.csv'],
    ['snnTorch', 'lif_snntorch.csv'],
    ['Spyx', 'lif_spyx.csv'],
    ['Lava (Loihi)', 'lif_lava_loihi_fixed.csv'],
    ['SpiNNaker2', 'lif_spinnaker2.csv'],
]
data = {
    name: np.loadtxt(filename, delimiter=',')
    for name, filename in filenames
}
SKIP_TIMESTEPS = 0
voltages = [
    (name, data[name][SKIP_TIMESTEPS:, 1] / data[name][:, 1].max()) for name in data.keys()
]
spikes = [
    (name, np.where(data[name][SKIP_TIMESTEPS:, 2] == 1)[0]) for name in data.keys()
]
input_data = data['Norse'][SKIP_TIMESTEPS:, 0]

In [2]:
!ls

debug_spike_representation  lif_lava_loihi_fixed.png  lif_rockpool.png
lif_comparison.ipynb	    lif_nengo.csv	      lif_sinabs.csv
lif_exact.csv		    lif_nengo.ipynb	      lif_sinabs.ipynb
lif_exact_sim.ipynb	    lif_nengo.png	      lif_snntorch.csv
lif_exact_sim.py	    lif_norse.csv	      lif_snntorch.ipynb
lif_lava_cpu_fixed.csv	    lif_norse.ipynb	      lif_spinnaker2.csv
lif_lava_cpu_fixed.png	    lif_norse.nir	      lif_spinnaker2.py
lif_lava_cpu_float.csv	    lif_norse.png	      lif_spyx.csv
lif_lava_cpu_float.png	    lif_rockpool.csv	      lif_spyx.ipynb
lif_lava.ipynb		    lif_rockpool.ipynb	      README.md
lif_lava_loihi_fixed.csv    lif_rockpool.nir


## 2D comparison plot

In [3]:
legend.get_frame().set_edgecolor

NameError: name 'legend' is not defined

In [ ]:
# plt.style.use("../plots.mplstyle")
# seaborn.set_context("paper", font_scale=1)
# plt.style.use("seaborn-v0_8-paper")
seaborn.set_style("whitegrid")
font = {'family' : 'CMU Sans Serif',
        'weight' : 'normal',
        'size'   : 20}

matplotlib.rc('font', **font)
matplotlib.rcParams["axes.edgecolor"] = "black"
matplotlib.rcParams["axes.linewidth"] = 1.5
matplotlib.rcParams["legend.edgecolor"] = "black"

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# fig.suptitle('Comparison of Leaky Integrate-and-Fire dynamics')
spec = mpl.gridspec.GridSpec(ncols=1, nrows=3, height_ratios=[1.5, 6, 2])
#fig = plt.figure(figsize=(12, 6), dpi=300)
fig, (ax0, ax1, ax3) = plt.subplots(3, 1, gridspec_kw={"height_ratios": [1, 6, 2], "hspace": 0.1}, sharex=True, figsize=(16, 6), dpi=300)

ax0.set_title("Single leaky integrate-and-fire neuron", pad=15, fontsize=30)

# ax0 = fig.add_subplot(spec[0])
ax0.set_ylabel('Input')
ax0.set_yticks([])
ax0.eventplot(np.where(input_data == 1.)[0], colors='k')
ax0.set_ylim(0, 2)
ax0.set_xlim(0, 1000)

# ax1 = fig.add_subplot(spec[1], sharex=ax0)
# handles = []
for idx, (name, voltage) in enumerate(voltages):
    if idx == 0:
        r = plt.Rectangle((1, -idx - 0.2), 998, 1.5, facecolor="#a0d6f2", edgecolor="black", linestyle="dotted")
        ax1.add_artist(r)
        offset = 0
    else:
        offset = -idx - 0.2
    ax1.plot(voltage * 0.85 + offset, label=name, linewidth=2)
    
ax1.set_ylabel('Voltage')
ax1.set_yticks([])
# ax1.set_yticklabels([])
ax1.grid(axis='y', alpha=0.5)

# ax3 = fig.add_subplot(spec[2], sharex=ax1)
ax3.set_ylabel('Spikes')
for idx, (name, spike) in enumerate(spikes[::-1]):
    yval = 1 + idx / (len(spikes)-1)
    ax3.scatter(spike, np.ones_like(spike) * yval, s=50, marker='+', color=colors[len(spikes) - idx - 1], linewidths=2)
ax3.set_ylim(0.9, 2.1)
ax3.set_yticklabels([])
ax3.set_yticks([])
ax3.grid(axis='y', alpha=0.5)
ax3.set_xlabel('Timestep', labelpad=-6)

legend = fig.legend(bbox_to_anchor=(0.87, -0.097, 0.2, 1), fontsize=20, labelspacing=0.73)
legend.get_frame().set_linewidth(1.5)
for l in legend.legend_handles:
    l.set_linewidth(2.5)
    
# fig.legend(bbox_to_anchor=(1, 1), ncols=len(voltages))
plt.tight_layout()
plt.savefig('../figures/lif_comparison.pdf', dpi=300, bbox_inches="tight")
plt.show()